<p style="font-size:small; color:gray;"> Author: 鄭永誠, Year: 2024 </p>

# 使用LangGraph實踐LLM Agent流程 - 2
----------

In [10]:
""" 讓print出來資訊變彩色的可愛套件 """
# %pip install termcolor -q


' 讓print出來資訊變彩色的可愛套件 '

### 1. 首先，我先設定了這邊要用的LLM模型
(當然，你也可以讓不同Agent使用不同模型唷! 也是種策略)

In [11]:
""" Setup LLM Model with GROQ API """


from typing import Annotated, Literal, TypedDict
import requests
from typing import List
import json
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq

END = '__end__'

# -*- coding: utf-8 -*-
class GroqModel:
    def __init__(self, temperature=0, model=None):
        self.api_key = "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"
        self.headers = {
            'Content-Type': 'application/json', 
            'Authorization': f'Bearer {self.api_key}'
            }
        self.model_endpoint = "https://api.groq.com/openai/v1/chat/completions"
        self.temperature = temperature
        self.model = model

    def invoke(self, messages):

        system = messages[0]["content"]
        user = messages[1]["content"]

        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": f"system:{system}\n\n user:{user}"
                }
            ],
            "temperature": self.temperature,
        }

        try:
            request_response = requests.post(
                self.model_endpoint, 
                headers=self.headers, 
                data=json.dumps(payload)
                )
            
            print("REQUEST RESPONSE", request_response)
            request_response_json = request_response.json()['choices'][0]['message']['content']
            response = str(request_response_json)
            
            response_formatted = HumanMessage(content=response)

            return response_formatted
        except requests.RequestException as e:
            response = {"error": f"Error in invoking model! {str(e)}"}
            response_formatted = HumanMessage(content=response)
            return response_formatted
        
class GroqJSONModel:
    def __init__(self, temperature=0, model=None):
        self.api_key = "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"
        self.headers = {
            'Content-Type': 'application/json', 
            'Authorization': f'Bearer {self.api_key}'
            }
        self.model_endpoint = "https://api.groq.com/openai/v1/chat/completions"
        self.temperature = temperature
        self.model = model

    def invoke(self, messages):

        system = messages[0]["content"]
        user = messages[1]["content"]

        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": f"system:{system}\n\n user:{user}"
                }
            ],
            "temperature": self.temperature,
            "response_format": {"type": "json_object"}
        }
        
        try:
            request_response = requests.post(
                self.model_endpoint, 
                headers=self.headers, 
                data=json.dumps(payload)
            )
            
            print("REQUEST RESPONSE", request_response.status_code)
            # print("REQUEST RESPONSE HEADERS", request_response.headers)
            # print("REQUEST RESPONSE TEXT", request_response.text)
            
            request_response_json = request_response.json()
            # print("REQUEST RESPONSE JSON", request_response_json)
            
            if 'choices' not in request_response_json or len(request_response_json['choices']) == 0:
                raise ValueError("No choices in response")

            response_content = request_response_json['choices'][0]['message']['content']
            # print("RESPONSE CONTENT", response_content)
            
            response = json.loads(response_content)
            response = json.dumps(response)

            response_formatted = HumanMessage(content=response)

            return response_formatted
        except (requests.RequestException, ValueError, KeyError) as e:
            error_message = f"Error in invoking model! {str(e)}"
            print("ERROR", error_message)
            response = {"error": error_message}
            response_formatted = HumanMessage(content=json.dumps(response))
            return response_formatted

### 2. 定義一個AgentGraphState，存儲所有 Agent 的狀態信息

In [12]:
""" 
建立 Agent Graph State，確認並記錄各 Agent 流程 
"""

from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# 定義 Agent Graph 的狀態對象
# 這裡使用 TypedDict 來定義字典結構，每個 Agent 的回應都以一個列表表示
class AgentGraphState(TypedDict):
    research_question: str  # 研究問題，表示當前要解決的問題或任務
    planner_response: Annotated[list, add_messages]  # 計劃者的回應
    selector_response: Annotated[list, add_messages]  # 選擇者的回應
    reporter_response: Annotated[list, add_messages]  # 報告者的回應
    reviewer_response: Annotated[list, add_messages]  # 審核者的回應
    router_response: Annotated[list, add_messages]  # 路由者的回應
    serper_response: Annotated[list, add_messages]  # 搜尋者的回應
    scraper_response: Annotated[list, add_messages]  # 爬取者的回應
    final_reports: Annotated[list, add_messages]  # 最終報告
    end_chain: Annotated[list, add_messages]  # 鏈結結束

# 定義獲取 Agent Graph 狀態的方法
# 這個方法根據給定的 state_key 返回對應的 Agent 回應數據
def get_agent_graph_state(state: AgentGraphState, state_key: str):
    if state_key == "planner_all":
        return state["planner_response"]  # 返回計劃者的所有回應
    elif state_key == "planner_latest":
        if state["planner_response"]:
            return state["planner_response"][-1]  # 返回計劃者的最新回應
        else:
            return state["planner_response"]  # 如果沒有回應，返回空列表

    elif state_key == "selector_all":
        return state["selector_response"]  # 返回選擇者的所有回應
    elif state_key == "selector_latest":
        if state["selector_response"]:
            return state["selector_response"][-1]  # 返回選擇者的最新回應
        else:
            return state["selector_response"]  # 如果沒有回應，返回空列表

    elif state_key == "reporter_all":
        return state["reporter_response"]  # 返回報告者的所有回應
    elif state_key == "reporter_latest":
        if state["reporter_response"]:
            return state["reporter_response"][-1]  # 返回報告者的最新回應
        else:
            return state["reporter_response"]  # 如果沒有回應，返回空列表

    elif state_key == "reviewer_all":
        return state["reviewer_response"]  # 返回審核者的所有回應
    elif state_key == "reviewer_latest":
        if state["reviewer_response"]:
            return state["reviewer_response"][-1]  # 返回審核者的最新回應
        else:
            return state["reviewer_response"]  # 如果沒有回應，返回空列表

    elif state_key == "serper_all":
        return state["serper_response"]  # 返回搜尋者的所有回應
    elif state_key == "serper_latest":
        if state["serper_response"]:
            return state["serper_response"][-1]  # 返回搜尋者的最新回應
        else:
            return state["serper_response"]  # 如果沒有回應，返回空列表

    elif state_key == "scraper_all":
        return state["scraper_response"]  # 返回爬取者的所有回應
    elif state_key == "scraper_latest":
        if state["scraper_response"]:
            return state["scraper_response"][-1]  # 返回爬取者的最新回應
        else:
            return state["scraper_response"]  # 如果沒有回應，返回空列表

    else:
        return None  # 如果 state_key 不匹配任何情況，返回 None
    
# 初始化 Agent Graph 的狀態
# 所有的回應列表都初始化為空列表，並且研究問題初始化為空字符串
state = {
    "research_question": "",
    "planner_response": [],
    "selector_response": [],
    "reporter_response": [],
    "reviewer_response": [],
    "router_response": [],
    "serper_response": [],
    "scraper_response": [],
    "final_reports": [],
    "end_chain": []
}

### 3. 一些工具和輔助function

In [13]:
""" 設定工具和其他一些輔助函數 """
import os

import json 
import requests
from bs4 import BeautifulSoup
from langchain_core.messages import HumanMessage
from datetime import datetime, timezone
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_groq.chat_models import ChatGroq
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import MessagesState
from langgraph.prebuilt.tool_node import ToolNode
from typing import List
from typing import Literal

END = '__end__'


# 亂碼的檢測函數
def is_garbled(text):
    # 簡單的檢測亂碼的啟發式方法：當非 ASCII 字符比例高判定為亂碼
    non_ascii_count = sum(1 for char in text if ord(char) > 127)
    return non_ascii_count > len(text) * 0.3

# 熟悉的美麗湯，用來抓取網頁內容

def scrape_website(state: AgentGraphState, research=None):
    research_data = research().content
    research_data = json.loads(research_data)
    # research_data = ast.literal_eval(research_data)

    try:
        url = research_data["selected_page_url"]
    except KeyError as e:
        url = research_data["error"]

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract text content
        texts = soup.stripped_strings
        content = ' '.join(texts)
        
        # Check for garbled text
        if is_garbled(content):
            content = "error in scraping website, garbled text returned"
        else:
            # Limit the content to 4000 characters
            content = content[:4000]

        state["scraper_response"].append(HumanMessage(role="system", content=str({"source": url, "content": content})))
        
        return {"scraper_response": state["scraper_response"]}
    
    except requests.HTTPError as e:
        if e.response.status_code == 403:
            content = f"error in scraping website, 403 Forbidden for url: {url}"
        else:
            content = f"error in scraping website, {str(e)}"
        
        state["scraper_response"].append(HumanMessage(role="system", content=str({"source": url, "content": content})))
        return {"scraper_response": state["scraper_response"]}
    except requests.RequestException as e:
        content = f"error in scraping website, {str(e)}"
        state["scraper_response"].append(HumanMessage(role="system", content=str({"source": url, "content": content})))
        return {"scraper_response": state["scraper_response"]}



# 用來獲取當前 UTC 日期和時間
def get_current_utc_datetime():
    now_utc = datetime.now(timezone.utc)
    current_time_utc = now_utc.strftime("%Y-%m-%d %H:%M:%S %Z")
    return current_time_utc

# 用來檢查狀態字典中的屬性是否有內容
def check_for_content(var):
    if var:
        try:
            var = var.content
            return var.content
        except:
            return var
    else:
        var

# 用於格式化搜索結果
def format_results(organic_results):

        result_strings = []
        for result in organic_results:
            title = result.get('title', 'No Title')
            link = result.get('link', '#')
            snippet = result.get('snippet', 'No snippet available.')
            result_strings.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}\n---")
        
        return '\n'.join(result_strings)

# 定義一個函數，用於從 Google Serper API 獲取搜索結果
# 注意，一樣要改成填入自己的 API 金鑰
def get_google_serper(state:AgentGraphState, plan):

    plan_data = plan().content
    plan_data = json.loads(plan_data)
    search = plan_data.get("search_term")

    # 從.env檔取得API key
    api_key = os.getenv('X-API-KEY')

    search_url = "https://google.serper.dev/search"
    headers = {
        'Content-Type': 'application/json',
        'X-API-KEY': api_key  
    }
    payload = json.dumps({"q": search})
    
    # Attempt to make the HTTP POST request
    try:
        response = requests.post(search_url, headers=headers, data=payload)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4XX, 5XX)
        results = response.json()
        
        # Check if 'organic' results are in the response
        if 'organic' in results:
            formatted_results = format_results(results['organic'])
            state = {**state, "serper_response": formatted_results}
            return state
        else:
            return {**state, "serper_response": "No organic results found."}

    except requests.exceptions.HTTPError as http_err:
        return {**state, "serper_response": f"HTTP error occurred: {http_err}"}
    except requests.exceptions.RequestException as req_err:
        return {**state, "serper_response": f"Request error occurred: {req_err}"}
    except KeyError as key_err:
        return {**state, "serper_response": f"Key error occurred: {key_err}"}



## 4. 設定各 Agent 的 system prompt 
1. **Planner（規劃者）:**
- 功能: 負責為研究問題制定一個全面的計劃，指導團隊如何有效地進行搜索。規劃者需根據收到的反饋調整計劃，並強調最相關的搜索詞，幫助其他團隊成員進行信息搜索。

- 輸出格式: JSON 格式，包含搜索詞、整體策略和附加信息。


2. **Selector（選擇者）**
- 功能: 負責從搜索引擎結果頁面中選擇最相關的搜索結果，並提供選擇的詳細原因。選擇者根據收到的反饋來調整選擇的結果。
- 輸出格式: JSON 格式，包含選擇的頁面URL、簡短描述和選擇原因。

3. **Reporter（報告者）**
- 功能: 負責根據所選擇的網頁內容，撰寫一個全面的回應，回答研究問題。報告者必須引用並參考信息來源，並根據反饋調整回應。
- 輸出格式: 結構化的文本回答，引用來源需標明URL。

4. **Reviewer（審核者）**
- 功能: 審核報告者的回應，並提供反饋。反饋應包括是否通過審核以及改進建議。審核者還需要考慮之前的代理工作結果。
- 輸出格式: JSON 格式，包含反饋意見、是否通過審核、是否全面、是否提供引用、是否與研究問題相關。

5. **Router（流程決策者）**
- 功能: 負責根據審核者提供的反饋決定下一步行動，選擇下一個應該接手任務的代理。可能的選擇包括規劃者、選擇者、報告者或進入最終報告階段。
- 輸出格式: JSON 格式，指定下一個代理。


這些角色之間協作，形成一個完整的工作流程，用於解決複雜的研究問題。每個角色在處理過程中都根據接收到的反饋進行調整和優化。


In [14]:
""" 設定各Agent的prompt"""
planner_prompt_template = """
你是一名規劃者。你的責任是制定一個全面的計劃來幫助你的團隊回答一個研究問題。
問題可能從簡單到複雜的多步驟查詢。你的計劃應該為你的團隊提供適當的指導，以有效地使用互聯網搜索引擎。

重點強調最相關的搜索詞開始，因為另一個團隊成員將使用你的建議搜索相關信息。

都要使用utf-8格式回答

如果你收到反饋，你必須相應地調整你的計劃。這裡是收到的反饋：
反饋：{feedback}

當前日期和時間：
{datetime}

你的回應必須採用以下 json 格式並用UTF-8編碼：

    "search_term": "最相關的搜索詞開始"
    "overall_strategy": "指導搜索過程的整體策略"
    "additional_information": "指導搜索的其他信息，包括其他搜索詞或過濾器"

"""

planner_guided_json = {
    "type": "object",
    "properties": {
        "search_term": {
            "type": "string",
            "description": "最相關的搜索詞開始"
        },
        "overall_strategy": {
            "type": "string",
            "description": "指導搜索過程的整體策略"
        },
        "additional_information": {
            "type": "string",
            "description": "指導搜索的其他信息，包括其他搜索詞或過濾器"
        }
    },
    "required": ["search_term", "overall_strategy", "additional_information"]
}


selector_prompt_template = """
你是一名選擇者。你將看到一個包含潛在相關搜索結果的搜索引擎結果頁面。你的任務是檢視所有結果，選擇最相關的一個，並提供選擇的詳細原因。

這是搜索引擎結果頁面：
{serp}

請以以下 json 格式返回你的發現，並用UTF-8編碼：

    "selected_page_url": "你選擇的頁面精確URL",
    "description": "頁面的簡短描述",
    "reason_for_selection": "你選擇此頁面的原因"

根據收到的反饋調整你的選擇：
反饋：{feedback}

這是你之前的選擇：
{previous_selections}
在做出新的選擇時請考慮這些信息。

當前日期和時間：
{datetime}
"""

selector_guided_json = {
    "type": "object",
    "properties": {
        "selected_page_url": {
            "type": "string",
            "description": "你選擇的頁面精確URL"
        },
        "description": {
            "type": "string",
            "description": "頁面的簡短描述"
        },
        "reason_for_selection": {
            "type": "string",
            "description": "你選擇此頁面的原因"
        }
    },
    "required": ["selected_page_url", "description", "reason_for_selection"]
}


reporter_prompt_template = """
你是一名報告者。你將看到一個包含與研究問題相關信息的網頁。你的任務是根據頁面上的信息提供一個全面的答案。確保引用和參考你的來源。

研究將以字典的形式呈現，來源為URL，內容為頁面上的文本：
研究：{research}

結構化你的回應如下：
根據收集到的信息，這是對查詢的全面回應：
"天空看起來是藍色的，因為一種稱為瑞利散射的現象，這種現象使得較短波長的光（藍色）比較長波長的光（紅色）散射更多 [1]。這種散射使天空大部分時間看起來是藍色的 [1]。此外，在日出和日落時，天空可能看起來是紅色或橙色，因為光線必須穿過更多的大氣層，將較短的藍色波長散射出視線，讓較長的紅色波長占主導地位 [2]。"

來源：
[1] https://example.com/science/why-is-the-sky-blue
[2] https://example.com/science/sunrise-sunset-colors

根據收到的反饋調整你的回應：
反饋：{feedback}

這是你之前的報告：
{previous_reports}

當前日期和時間：
{datetime}
"""


reviewer_prompt_template = """
你是一名審核者。你的任務是審核報告者對研究問題的回應並提供反饋。

這是報告者的回應：
報告者的回應：{reporter}

你的反饋應包括通過或未通過審核的原因和改進建議。

在提供新反饋時應考慮你之前給出的反饋。
反饋：{feedback}

當前日期和時間：
{datetime}

你應該了解之前代理的所做工作。你可以在代理狀態中看到這些信息：
代理狀態：{state}

你的回應必須採用以下 json 格式，並用UTF-8編碼：

    "feedback": "如果回應未通過審核，請提供具體反饋以便通過審核。",
    "pass_review": "True/False",
    "comprehensive": "True/False",
    "citations_provided": "True/False",
    "relevant_to_research_question": "True/False",

"""


reviewer_guided_json = {
    "type": "object",
    "properties": {
        "feedback": {
            "type": "string",
            "description": "你的反饋。說明為什麼你通過或未通過審核"
        },
        "pass_review": {
            "type": "boolean",
            "description": "True/False"
        },
        "comprehensive": {
            "type": "boolean",
            "description": "True/False"
        },
        "citations_provided": {
            "type": "boolean",
            "description": "True/False"
        },
        "relevant_to_research_question": {
            "type": "boolean",
            "description": "True/False"
        }
    },
    "required": ["feedback", "pass_review", "comprehensive", "citations_provided", "relevant_to_research_question"]
}

router_prompt_template = """
你是一名流程決策者。你的任務是根據審核者提供的反饋將對話決策到下一個代理。你必須選擇以下代理之一：規劃者、選擇者、報告者或最終報告。

這是審核者提供的反饋：
反饋：{feedback}

### 選擇下一個代理的標準：
- **規劃者**：如果需要新信息。
- **選擇者**：如果需要選擇不同的來源。
- **報告者**：如果報告的格式或風格需要改進，或如果回應缺乏清晰性或全面性。
- **最終報告**：如果反饋標記為通過審核（pass_review）為True，你必須選擇最終報告。

你必須以以下 json 格式提供你的回應，並用UTF-8編碼：
    
        "next_agent": "規劃者/選擇者/報告者/最終報告 之一"
    
"""

## 5. 建立各個Agent (class)

In [15]:
""" 建立各個Agent """
from termcolor import colored
import json
def handle_encoding(text):
    # 將 JSON 字符串轉換成 Python 字典
    try:
        data = json.loads(text)
        return data
    except:
        return text


class Agent:
    def __init__(self, state: AgentGraphState, model=None, server=None, temperature=0, model_endpoint=None, stop=None, guided_json=None):
        self.state = state
        self.model = model
        self.server = server
        self.temperature = temperature
        self.model_endpoint = model_endpoint
        self.stop = stop
        self.guided_json = guided_json

    def get_llm(self, json_model=True):

        if self.server == 'groq':
            return GroqJSONModel(
                model=self.model,
                temperature=self.temperature
            ) if json_model else GroqModel(
                model=self.model,
                temperature=self.temperature
            )


    def update_state(self, key, value):
        self.state = {**self.state, key: value}

class PlannerAgent(Agent):
    def invoke(self, research_question, prompt=planner_prompt_template, feedback=None):
        feedback_value = feedback() if callable(feedback) else feedback
        feedback_value = check_for_content(feedback_value)

        planner_prompt = prompt.format(
            feedback=feedback_value,
            datetime=get_current_utc_datetime()
        )

        messages = [
            {"role": "system", "content": planner_prompt},
            {"role": "user", "content": f"research question: {research_question}"}
        ]

        llm = self.get_llm()
        ai_msg = llm.invoke(messages)
        response = ai_msg.content
        show_response = handle_encoding(response)

        self.update_state("planner_response", response)
        print(colored(f"規劃者 Planner 👩🏿‍💻:\n {show_response}", 'cyan'))
        return self.state

class SelectorAgent(Agent):
    def invoke(self, research_question, prompt=selector_prompt_template, feedback=None, previous_selections=None, serp=None):
        feedback_value = feedback() if callable(feedback) else feedback
        previous_selections_value = previous_selections() if callable(previous_selections) else previous_selections

        feedback_value = check_for_content(feedback_value)
        previous_selections_value = check_for_content(previous_selections_value)

        selector_prompt = prompt.format(
            feedback=feedback_value,
            previous_selections=previous_selections_value,
            serp=serp().content,
            datetime=get_current_utc_datetime()
        )

        messages = [
            {"role": "system", "content": selector_prompt},
            {"role": "user", "content": f"research question: {research_question}"}
        ]

        llm = self.get_llm()
        ai_msg = llm.invoke(messages)
        response = ai_msg.content
        show_response = handle_encoding(response)

        print(colored(f"選擇者 Selector 🧑🏼‍💻:\n {show_response}", 'green'))
        self.update_state("selector_response", response)
        return self.state

class ReporterAgent(Agent):
    def invoke(self, research_question, prompt=reporter_prompt_template, feedback=None, previous_reports=None, research=None):
        feedback_value = feedback() if callable(feedback) else feedback
        previous_reports_value = previous_reports() if callable(previous_reports) else previous_reports
        research_value = research() if callable(research) else research

        feedback_value = check_for_content(feedback_value)
        previous_reports_value = check_for_content(previous_reports_value)
        research_value = check_for_content(research_value)
        
        reporter_prompt = prompt.format(
            feedback=feedback_value,
            previous_reports=previous_reports_value,
            datetime=get_current_utc_datetime(),
            research=research_value
        )

        messages = [
            {"role": "system", "content": reporter_prompt},
            {"role": "user", "content": f"research question: {research_question}"}
        ]

        llm = self.get_llm(json_model=False)
        ai_msg = llm.invoke(messages)
        response = ai_msg.content
        show_response = handle_encoding(response)

        print(colored(f"報告者 Reporter 👨‍💻:\n {show_response}", 'yellow'))
        self.update_state("reporter_response", response)
        return self.state

class ReviewerAgent(Agent):
    def invoke(self, research_question, prompt=reviewer_prompt_template, reporter=None, feedback=None):
        reporter_value = reporter() if callable(reporter) else reporter
        feedback_value = feedback() if callable(feedback) else feedback

        reporter_value = check_for_content(reporter_value)
        feedback_value = check_for_content(feedback_value)
        
        reviewer_prompt = prompt.format(
            reporter=reporter_value,
            state=self.state,
            feedback=feedback_value,
            datetime=get_current_utc_datetime(),
        )

        messages = [
            {"role": "system", "content": reviewer_prompt},
            {"role": "user", "content": f"research question: {research_question}"}
        ]

        llm = self.get_llm()
        ai_msg = llm.invoke(messages)
        response = ai_msg.content
        show_response = handle_encoding(response)

        print(colored(f"查核者 Reviewer 👩🏽‍⚖️:\n {show_response}", 'magenta'))
        self.update_state("reviewer_response", response)
        return self.state
    
class RouterAgent(Agent):
    def invoke(self, feedback=None, research_question=None, prompt=router_prompt_template):
        feedback_value = feedback() if callable(feedback) else feedback
        feedback_value = check_for_content(feedback_value)

        router_prompt = prompt.format(feedback=feedback_value)

        messages = [
            {"role": "system", "content": router_prompt},
            {"role": "user", "content": f"research question: {research_question}"}
        ]

        llm = self.get_llm()
        ai_msg = llm.invoke(messages)
        response = ai_msg.content
        show_response = handle_encoding(response)

        print(colored(f"指派者 Router 🧭:\n {show_response}", 'blue'))
        self.update_state("router_response", response)
        return self.state

class FinalReportAgent(Agent):
    def invoke(self, final_response=None):
        final_response_value = final_response() if callable(final_response) else final_response
        response = final_response_value.content
        show_response = handle_encoding(response)

        print(colored(f"Final Report 📝:\n {show_response}", 'blue'))
        self.update_state("final_reports", response)
        return self.state

class EndNodeAgent(Agent):
    def invoke(self):
        self.update_state("end_chain", "end_chain")
        return self.state

## 6. 建立 Agent Graph 間關聯的流程圖
- 用add_node方法添加節點  

- 用add_edge增加節點間流程

- 用add_conditional_edges增加節點間條件流程

- 用set_entry_point設定開始節點

- 用set_finish_point設定結束節點


In [16]:
""" Agent Graph """
import json
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

router_guided_json = {
    "type": "object",
    "properties": {
        "next_agent": {
            "type": "string",
            "description": "one of the following: planner/selector/reporter/final_report"
        }
    },
    "required": ["next_agent"]
}


def create_graph(server=None, model=None, stop=None, model_endpoint=None, temperature=0):
    graph = StateGraph(AgentGraphState)

    graph.add_node(
        "planner", 
        lambda state: PlannerAgent(
            state=state,
            model=model,
            server=server,
            guided_json=planner_guided_json,
            stop=stop,
            model_endpoint=model_endpoint,
            temperature=temperature
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(state=state, state_key="reviewer_latest"),
            # previous_plans=lambda: get_agent_graph_state(state=state, state_key="planner_all"),
            prompt=planner_prompt_template
        )
    )

    graph.add_node(
        "selector",
        lambda state: SelectorAgent(
            state=state,
            model=model,
            server=server,
            guided_json=selector_guided_json,
            stop=stop,
            model_endpoint=model_endpoint,
            temperature=temperature
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(state=state, state_key="reviewer_latest"),
            previous_selections=lambda: get_agent_graph_state(state=state, state_key="selector_all"),
            serp=lambda: get_agent_graph_state(state=state, state_key="serper_latest"),
            prompt=selector_prompt_template,
        )
    )

    graph.add_node(
        "reporter", 
        lambda state: ReporterAgent(
            state=state,
            model=model,
            server=server,
            stop=stop,
            model_endpoint=model_endpoint,
            temperature=temperature
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(state=state, state_key="reviewer_latest"),
            previous_reports=lambda: get_agent_graph_state(state=state, state_key="reporter_all"),
            research=lambda: get_agent_graph_state(state=state, state_key="scraper_latest"),
            prompt=reporter_prompt_template
        )
    )

    graph.add_node(
        "reviewer", 
        lambda state: ReviewerAgent(
            state=state,
            model=model,
            server=server,
            guided_json=reviewer_guided_json,
            stop=stop,
            model_endpoint=model_endpoint,
            temperature=temperature
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(state=state, state_key="reviewer_all"),
            reporter=lambda: get_agent_graph_state(state=state, state_key="reporter_latest"),
            prompt=reviewer_prompt_template
        )
    )

    graph.add_node(
        "router", 
        lambda state: RouterAgent(
            state=state,
            model=model,
            server=server,
            guided_json=router_guided_json,
            stop=stop,
            model_endpoint=model_endpoint,
            temperature=temperature
        ).invoke(
            research_question=state["research_question"],
            feedback=lambda: get_agent_graph_state(state=state, state_key="reviewer_all"),
            prompt=router_prompt_template
        )
    )


    graph.add_node(
        "serper_tool",
        lambda state: get_google_serper(
            state=state,
            plan=lambda: get_agent_graph_state(state=state, state_key="planner_latest")
        )
    )

    graph.add_node(
        "scraper_tool",
        lambda state: scrape_website(
            state=state,
            research=lambda: get_agent_graph_state(state=state, state_key="selector_latest")
        )
    )

    graph.add_node(
        "final_report", 
        lambda state: FinalReportAgent(
            state=state
        ).invoke(
            final_response=lambda: get_agent_graph_state(state=state, state_key="reporter_latest")
        )
    )

    graph.add_node("end", lambda state: EndNodeAgent(state).invoke())

    # Define the edges in the agent graph
    def pass_review(state: AgentGraphState):
        review_list = state["router_response"]
        if review_list:
            review = review_list[-1]
        else:
            review = "No review"

        if review != "No review":
            if isinstance(review, HumanMessage):
                review_content = review.content
            else:
                review_content = review
            
            review_data = json.loads(review_content)
            next_agent = review_data["next_agent"]
        else:
            next_agent = "end"

        return next_agent

    # Add edges to the graph
    graph.set_entry_point("planner")
    graph.set_finish_point("end")
    graph.add_edge("planner", "serper_tool")
    graph.add_edge("serper_tool", "selector")
    graph.add_edge("selector", "scraper_tool")
    graph.add_edge("scraper_tool", "reporter")
    graph.add_edge("reporter", "reviewer")
    graph.add_edge("reviewer", "router")

    graph.add_conditional_edges(
        "router",
        lambda state: pass_review(state=state),
    )

    graph.add_edge("final_report", "end")
    compiled_graph = graph.compile()
    # display(Image(compiled_graph.get_graph().draw_mermaid_png()))
    return graph

In [17]:
""" 定義一個建立 Agent Graph 並啟動的函式 """
def compile_workflow(graph):
    workflow = graph.compile()
    return workflow

## 實際操作範例

In [18]:
# -*- coding: utf-8 -*-

# 定義使用模型相關參數
verbose = False
iterations = 30
server = 'groq'
model = 'llama-3.1-70b-versatile'
model_endpoint = None

# call create_graph and compile_workflow
graph = create_graph(server=server, model=model, model_endpoint=model_endpoint)
workflow = compile_workflow(graph)


while True:
    query = input("Please enter your research question: ")
    if query.lower() == "exit":
        break

    dict_inputs = {"research_question": query}
    # thread = {"configurable": {"thread_id": "紀錄的thread_id"}}
    limit = {"recursion_limit": iterations}

    
    for event in workflow.stream(
        dict_inputs, limit
        ):
        if verbose:
            print("\nState Dictionary:", event)
        else:
            print("\n")

# 執行後就可以輸入問句去問問題了
# 問題範例: 下屆奧運辦在哪，為甚麼要辦在那裏?
# 問題範例: 這個國家擅長甚麼運動項目?



REQUEST RESPONSE 524
ERROR Error in invoking model! Expecting value: line 1 column 1 (char 0)
規劃者 Planner 👩🏿‍💻:
 {'error': 'Error in invoking model! Expecting value: line 1 column 1 (char 0)'}




REQUEST RESPONSE 524
ERROR Error in invoking model! Expecting value: line 1 column 1 (char 0)
選擇者 Selector 🧑🏼‍💻:
 {'error': 'Error in invoking model! Expecting value: line 1 column 1 (char 0)'}




